In [3]:
pip install --upgrade joblib


Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import joblib

cat_features = ['basvuru_yili',
 'cinsiyet',
 'dogum_yeri',
 'ikametgah_sehri',
 'universite_adi',
 'universite_turu',
 'burslu_ise_burs_yuzdesi',
 'burs_aliyor_mu?',
 'bolum',
 'universite_kacinci_sinif',
 'universite_not_ortalamasi',
 'daha_once_baska_bir_universiteden_mezun_olmus',
 'lise_adi',
 'lise_adi_diger',
 'lise_sehir',
 'lise_turu',
 'lise_bolumu',
 'lise_bolum_diger',
 'lise_mezuniyet_notu',
 'baska_bir_kurumdan_burs_aliyor_mu?',
 'burs_aldigi_baska_kurum',
 'baska_kurumdan_aldigi_burs_miktari',
 'anne_egitim_durumu',
 'anne_calisma_durumu',
 'baba_egitim_durumu',
 'baba_calisma_durumu',
 'kardes_sayisi',
 'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
 'uye_oldugunuz_kulubun_ismi',
 'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
 'spor_dalindaki_rolunuz_nedir?',
 'aktif_olarak_bir_stk_uyesi_misiniz?',
 "hangi_stk'nin_uyesisiniz?",
 'stk_projesine_katildiniz_mi?',
 'girisimcilikle_ilgili_deneyiminiz_var_mi?',
 'girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz?',
 'ingilizce_biliyor_musunuz?',
 'ingilizce_seviyeniz?',
 'daha_onceden_mezun_olunduysa,_mezun_olunan_universite',
 'id',
 'anne_sektor_encoded',
 'baba_sektor_encoded',
 'anne_diger',
 'anne_kamu',
 'anne_Unknown',
 'anne_ozel sektor',
 'baba_diger',
 'baba_kamu',
 'baba_Unknown',
 'baba_ozel sektor',
 'age']

# Load the saved models
clf_pipeline = joblib.load('classification_model.pkl')
reg_model = joblib.load('regression_model.pkl')
scaler = joblib.load('scaler.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')  # Load the saved TF-IDF vectorizer

# Load the Word2Vec model
word2vec_model = Word2Vec.load('word2vec_model.pkl')

# Load the test data
test_df = pd.read_csv('cleanedTest.csv')

# Handle missing values in text columns
text_columns = ['girisimcilikle_ilgili_deneyiminiz_var_mi?', 'girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz?']
test_df[text_columns] = test_df[text_columns].fillna('')

# Combine text columns into one for processing
test_df['combined_text'] = test_df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Tokenize and preprocess text data for Word2Vec
def preprocess_text(text):
    return simple_preprocess(text, deacc=True)

# Create document vectors by averaging word vectors
def vectorize_text(text):
    tokens = preprocess_text(text)
    vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    if len(vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(vectors, axis=0)

# Vectorize text data
test_df['text_vector'] = test_df['combined_text'].apply(vectorize_text)
word2vec_features_test = np.array(test_df['text_vector'].tolist())

# Vectorize test data with the fitted TF-IDF vectorizer
tfidf_vectors_test = tfidf_vectorizer.transform(test_df['combined_text'])

# Combine Word2Vec and TF-IDF features
combined_features_test = hstack([tfidf_vectors_test, word2vec_features_test])


# Prepare data for classification model
X_cat_test = test_df[cat_features]  # Ensure cat_features is defined and matches your training data

# Predict using classification model
y_pred_cat = clf_pipeline.predict(X_cat_test)
print("Classification Predictions:\n", y_pred_cat)

# Prepare data for regression model
X_combined_test = combined_features_test
X_combined_test = scaler.transform(X_combined_test)

# Predict using regression model
y_pred_reg = reg_model.predict(X_combined_test)
print("Regression Predictions:\n", y_pred_reg)


Classification Predictions:
 [24.  7. 14. ... 33. 34.  7.]
Regression Predictions:
 [25.311255   25.311255   25.311255   ... 25.311255   32.06822074
 25.311255  ]


In [62]:
list = range(0,11049)
submit = pd.DataFrame(list)
submit["id"] = list
submit["Degerlendirme Puani"] =  y_pred_cat
submit = submit.drop(0,axis = 1)
submit

,id,Degerlendirme Puani
0,0,24.0
1,1,7.0
2,2,14.0
3,3,33.0
4,4,17.0
...,...,...
11044,11044,33.0
11045,11045,33.0
11046,11046,33.0
11047,11047,34.0


In [64]:
submit.to_csv("submit1.csv",index =False)